In [ ]:
# Necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# For matrix factorization
from scipy.sparse.linalg import svds
!pip install surprise
import pandas as pd
!pip install openpyxl
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
pd.set_option('display.max_columns', None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 771 kB 21.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626479 sha256=47bf935ba9ed0ab703f3397404f183c6bfa1b2057b772c8c04d9fa1491036985
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ratings = '/content/drive/MyDrive/ColabNotebooks/Small/ratings.csv'
movies = '/content/drive/MyDrive/ColabNotebooks/Small/movies.csv'
tags = '/content/drive/MyDrive/ColabNotebooks/Small/tags.csv'
links = '/content/drive/MyDrive/ColabNotebooks/Small/links.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load movies dataset that contains movie information with attributes like movieId, title and genres
movies_df = pd.read_csv(movies)
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# Checking the dimensionality of movies_df
movies_df.shape

(9742, 3)

In [ ]:
# Load ratings dataset that contains ratings information with attributes like userId, movieId, rating and timestamp
ratings_df = pd.read_csv(ratings)
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
# Checking the dimensionality of ratings_df
ratings_df.shape


(100836, 4)

In [ ]:
#Merging movies_df and ratings_df into one df on attribute movieId

df = movies_df.merge(ratings_df, how="left", on="movieId")
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [ ]:
#Taking a much smaller dataset by selecting a few movies at random

movie_ids = [130219, 356, 4422, 541, 368, 484, 540, 987, 1090, 100044, 155168]
movies = ["The Dark Knight (2011)",
          "Cries and Whispers (Viskningar och rop) (1972)",
          "Forrest Gump (1994)",
          "Blade Runner (1982)",
          "Maverick (1994)",
          "Lassie (1994)",
          "Sliver (1993)",
          "Bliss (1997)",
          "Platoon (1986)",
          "Human Planet (2011)",
          "Fraktus (2012)"]

In [ ]:
randomsample_df = df[df.movieId.isin(movie_ids)]
randomsample_df.shape

(615, 6)

In [ ]:
randomsample_df.head()

,movieId,title,genres,userId,rating,timestamp
10019,356,Forrest Gump (1994),Comedy|Drama|Romance|War,1.0,4.0,9.649810e+08
10020,356,Forrest Gump (1994),Comedy|Drama|Romance|War,6.0,5.0,8.455532e+08
10021,356,Forrest Gump (1994),Comedy|Drama|Romance|War,7.0,5.0,1.106636e+09
10022,356,Forrest Gump (1994),Comedy|Drama|Romance|War,8.0,3.0,8.394635e+08
10023,356,Forrest Gump (1994),Comedy|Drama|Romance|War,10.0,3.5,1.455302e+09


In [ ]:
#Creating a user_movie_df by pivoting on userId to create a user-item matrix with users in rows and movies in columns.

user_movie_df = randomsample_df.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df.shape

(380, 10)

In [ ]:
user_movie_df.head()

title,Blade Runner (1982),Bliss (1997),Cries and Whispers (Viskningar och rop) (1972),Forrest Gump (1994),Fraktus (2012),Human Planet (2011),Lassie (1994),Maverick (1994),Platoon (1986),Sliver (1993)
userId,,,,,,,,,,
1.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
6.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,NaN,3.0
7.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
8.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
 #Using the reader object we scale the ratings to the range of 1-5
reader = Reader(rating_scale=(1, 5))

In [ ]:
input_data = Dataset.load_from_df(randomsample_df[['userId', 'movieId', 'rating']], reader)

In [ ]:
#Splitting input data into train and test dataset and train the trainset with the singular value decomposition model

train_set, test_set = train_test_split(input_data, test_size=.25)
svd_model = SVD()
svd_model.fit(train_set)

In [ ]:
#Procure predictions using the trained SVD model and get predictions. Accuracy for SVD models is best determined by RMSE

predictions_per_user = svd_model.test(test_set)

accuracy.rmse(predictions_per_user)

RMSE: 0.9002


0.9002376332731964

In [ ]:
# Applying cross validation method in order to validate the prediction results. Evaluating model performance with RMSE and MAE with K=5 on 5 split(s).

cross_validate(svd_model, input_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8464  0.8539  0.9711  0.8597  0.9153  0.8893  0.0476  
MAE (testset)     0.6475  0.6799  0.7287  0.6998  0.7287  0.6969  0.0308  
Fit time          0.02    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([0.84637046, 0.85389411, 0.9710612 , 0.85966584, 0.91533225]),
 'test_mae': array([0.64751342, 0.67985636, 0.72870504, 0.69975558, 0.72865228]),
 'fit_time': (0.01606893539428711,
  0.00853276252746582,
  0.007600307464599609,
  0.007218837738037109,
  0.007088422775268555),
 'test_time': (0.0015933513641357422,
  0.0008707046508789062,
  0.0008563995361328125,
  0.0008039474487304688,
  0.0007932186126708984)}

In [ ]:
# Using the svd_model we built to determine if for a given user and movie id combination, the model can give us an appropriate prediction. 
# For userId=3 and movie ID=368 ("Maverick (1994)), let the model give a suggestion.

svd_model.predict(uid=3.0, iid=368, verbose=True)

user: 3.0        item: 368        r_ui = None   est = 3.63   {'was_impossible': False}


Prediction(uid=3.0, iid=368, r_ui=None, est=3.6288882953446215, details={'was_impossible': False})

# Basically, this user didn't rate the movie with Id = 368 but we estimated it as 3.63.

In [ ]:
 #Using GridSearchCv algorithm to tune the model

param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005]}

gs = GridSearchCV(SVD,
                  param_grid,
                  measures=['rmse', 'mae'],
                  cv=3,
                  n_jobs=-1,
                  joblib_verbose=True)

gs.fit(input_data)
gs.best_score['rmse']

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.2s finished


0.892761840397684

Before tuning, the rmse was 0.89.

In [ ]:
gs.best_params['rmse']

{'n_epochs': 10, 'lr_all': 0.005}

In [ ]:
svd_model = SVD(**gs.best_params['rmse'])

data = input_data.build_full_trainset()
svd_model.fit(data)

svd_model.predict(uid=3.0, iid=368, verbose=True)

user: 3.0        item: 368        r_ui = None   est = 3.53   {'was_impossible': False}


Prediction(uid=3.0, iid=368, r_ui=None, est=3.5260660693346777, details={'was_impossible': False})

Previously, the estimated score for Maverick(1994) for user ID=3 was 3.63 and we tuned our SVD model to get an estimated score of 3.53